<a href="https://colab.research.google.com/github/Noman-ashraf1/TextSummarizer/blob/main/Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#Load libraries


In [2]:
import numpy as np
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split



#Load dataset

In [3]:
df = pd.read_csv("/content/News_summarisation_Dataset_BBC_11Nov2024 (1).csv")
df.drop(columns=['link'],axis=1,inplace=True)
df.head()

,headline,content
0,Russian gains accelerate as Ukraine's Kursk ga...,President Biden’s decision to provide anti-per...
1,Trump picks Linda McMahon and Mehmet Oz to ser...,US President-elect Donald Trump has picked Wor...
2,Russian gains accelerate as Ukraine's Kursk ga...,President Biden’s decision to provide anti-per...
3,Trump picks Linda McMahon and Mehmet Oz to ser...,US President-elect Donald Trump has picked Wor...
4,Logan Paul accused of misleading fans over cry...,"Logan Paul, the massively popular social media..."


In [4]:
df['summerization'] = df['headline']
df.drop(columns=['headline'],axis=1,inplace=True)


In [5]:
df.head()

,content,summerization
0,President Biden’s decision to provide anti-per...,Russian gains accelerate as Ukraine's Kursk ga...
1,US President-elect Donald Trump has picked Wor...,Trump picks Linda McMahon and Mehmet Oz to ser...
2,President Biden’s decision to provide anti-per...,Russian gains accelerate as Ukraine's Kursk ga...
3,US President-elect Donald Trump has picked Wor...,Trump picks Linda McMahon and Mehmet Oz to ser...
4,"Logan Paul, the massively popular social media...",Logan Paul accused of misleading fans over cry...


#Data preprocessing

In [6]:
import re
def clean_text(text):
    text = text.strip().lower()  # Remove leading/trailing spaces and convert to lowercase
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters  # Split into words (creates a list)
    return text
df['content'] = df['content'].apply(clean_text)
df['summerization']= df['summerization'].apply(clean_text)

In [7]:
df.head()

,content,summerization
0,president bidens decision to provide antiperso...,russian gains accelerate as ukraines kursk gam...
1,us presidentelect donald trump has picked worl...,trump picks linda mcmahon and mehmet oz to ser...
2,president bidens decision to provide antiperso...,russian gains accelerate as ukraines kursk gam...
3,us presidentelect donald trump has picked worl...,trump picks linda mcmahon and mehmet oz to ser...
4,logan paul the massively popular social media ...,logan paul accused of misleading fans over cry...


#Split the data

In [8]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
train_df.shape

(38, 2)

In [10]:
from datasets import Dataset
# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(val_df)

#Tokenization

In [11]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
from transformers import AutoTokenizer
import tensorflow
# Assuming tokenizer has been defined
def process_fun(example):
    inputs = tokenizer(example['content'], padding='max_length', truncation=True, max_length=512)
    output = tokenizer(example['summerization'], padding='max_length', truncation=True, max_length=512)
    inputs['label'] = output['input_ids']
    return inputs

# Apply the process function
train_data = train_dataset.map(process_fun, batched=True)
val_data = test_dataset.map(process_fun, batched=True)


Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

#Fine tune the model

In [71]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Directory to save model checkpoints
    num_train_epochs=10,                  # Number of epochs
    per_device_train_batch_size=8,        # Batch size for training
    per_device_eval_batch_size=8,         # Batch size for evaluation
    warmup_steps=500,                     # Warmup steps for learning rate scheduler
    weight_decay=0.01,                    # Weight decay for optimizer
    logging_dir="./logs",                 # Directory for logs
    evaluation_strategy="steps",          # Evaluate every 'eval_steps'
    eval_steps=100,                       # Number of steps to evaluate
    save_strategy="epoch",                # Save model checkpoints at the end of each epoch
    logging_steps=100,                    # Log every 100 steps
    learning_rate=5e-5,                   # Learning rate
    load_best_model_at_end=False,         # Do not load the best model at the end
    report_to="tensorboard"               # Log to TensorBoard
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-71-60916a9123da>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


TrainOutput(global_step=50, training_loss=10.986622314453125, metrics={'train_runtime': 96.8175, 'train_samples_per_second': 3.925, 'train_steps_per_second': 0.516, 'total_flos': 51429884559360.0, 'train_loss': 10.986622314453125, 'epoch': 10.0})

#Prediction

In [72]:
from transformers import pipeline

summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

# Generate predictions on validation set
predictions = [summarization_pipeline(example['content'])[0]['summary_text'] for example in val_data]




Device set to use cuda:0


In [73]:
new_texts = ["ariana grande has told the bbc that she channelled her personal feelings of loss when filming wickedlosing someone you love is something weve all unfortunately had to experience  and sometimes we have the privilege to say goodbye and sometimes we dont she saysgrande 31 plays galinda upland in the film which is an adaptation of the hugely successful stage musical exploring the wizard of oz universe from the perspective of two witchesthe twotime grammy award winner has suffered personal tragedy in recent years after the2017 manchester bombing of her concertand thedeath of her former partner mac millera year later she says appearing in wicked one of the first broadway shows she saw as a child feels like a homecomingthis music has always brought such comfort and now being able to spend time with it and be trusted with it is the privilege of a lifetimein the weeks running up to the films release the close relationship between grande and costar cynthia erivo has been in the spotlightfrom the"]
generated_summaries = summarization_pipeline(new_texts)

for i, summary in enumerate(generated_summaries):
    print(f"Original: {new_texts[i]}")
    print(f"Summary: {summary['summary_text']}")


Original: ariana grande has told the bbc that she channelled her personal feelings of loss when filming wickedlosing someone you love is something weve all unfortunately had to experience  and sometimes we have the privilege to say goodbye and sometimes we dont she saysgrande 31 plays galinda upland in the film which is an adaptation of the hugely successful stage musical exploring the wizard of oz universe from the perspective of two witchesthe twotime grammy award winner has suffered personal tragedy in recent years after the2017 manchester bombing of her concertand thedeath of her former partner mac millera year later she says appearing in wicked one of the first broadway shows she saw as a child feels like a homecomingthis music has always brought such comfort and now being able to spend time with it and be trusted with it is the privilege of a lifetimein the weeks running up to the films release the close relationship between grande and costar cynthia erivo has been in the spotlig

#Example

In [74]:
predictions = ["new_texts"]

# Multiple references (each reference is a list containing ground-truth summaries)
references = [
    ["generated_summaries"]
]


In [75]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu


bleu_score = corpus_bleu(references, predictions)
print(f"BLEU Score: {bleu_score}")



BLEU Score: 3.1375089891669103e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
